In [1]:
import numpy as np
import numpy.linalg as la
import copy
# Define the matrix A
A = np.array([[1, 2, 3],
              [0, 1, 4]])

In [2]:
S1 = A @ A.T

In [3]:
S2 = np.zeros_like(S1)
for i in range(A.shape[0]):
    S2 += np.outer(A[:, i], A[:, i])

In [4]:
def grahm_schmidt(A):
    """Perform Gram-Schmidt orthogonalization on the columns of A."""
    Q = np.zeros_like(A)
    for i in range(A.shape[1]):
        v = A[:, i]
        for j in range(i):
            v -= np.dot(Q[:, j], A[:, i]) * Q[:, j]
        Q[:, i] = v / la.norm(v)
    return Q

In [5]:
import numpy as np
from faces_places import is_orthonormal
def gram_schmidt(A):
    """Compute an orthogonal matrix using the Gram-Schmidt process on the rows of A"""

    Q = np.ones_like(A, dtype=float)
    print(f'Q is {Q}')
    print(Q.shape)
    for i in range(1, A.shape[0]):
        u = A[i]
        for v in Q:
            projuv = v * np.dot(u, v) / np.dot(v, v)
            print('u is: ', u)
            print('v is: ', v)
            print('projuv is: ', projuv)
            u -= projuv
        Q[i] = u
    return Q
A = np.array([[1, 2, 3],
              [0, 1, 4]], dtype=float)
Q = gram_schmidt(A)

is_orthonormal(Q)

Q is [[1. 1. 1.]
 [1. 1. 1.]]
(2, 3)
u is:  [0. 1. 4.]
v is:  [1. 1. 1.]
projuv is:  [1.66666667 1.66666667 1.66666667]
u is:  [-1.66666667 -0.66666667  2.33333333]
v is:  [1. 1. 1.]
projuv is:  [-1.48029737e-16 -1.48029737e-16 -1.48029737e-16]
Matrix shape: 2 rows, 3 columns
Row orthogonality: yes
Row normality: no
Column orthogonality: trivially no
Column normality: no


In [6]:
Q = gram_schmidt(A)

Q is [[1. 1. 1.]
 [1. 1. 1.]]
(2, 3)
u is:  [-1.66666667 -0.66666667  2.33333333]
v is:  [1. 1. 1.]
projuv is:  [-3.70074342e-17 -3.70074342e-17 -3.70074342e-17]
u is:  [-1.66666667 -0.66666667  2.33333333]
v is:  [1. 1. 1.]
projuv is:  [0. 0. 0.]


In [7]:
def is_diag(mat, tol=1e-4):
    """
    Returns boolean answer to whether or not mat is a diagonal matrix
    :param mat:
    :param tol:
    :return:
    """
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            if i != j and mat[i, j] > tol:
                return False

    return True

def test_is_diag():
    import numpy as np

    # Diagonal matrix
    mat1 = np.diag([1, 2, 3])
    assert is_diag(mat1)

    # Non-diagonal matrix
    mat2 = np.array([[1, 2], [0, 1]])
    assert not is_diag(mat2)

    # Nearly diagonal (off-diagonal below tolerance)
    mat3 = np.array([[1, 1e-5], [0, 2]])
    assert is_diag(mat3)

    # Nearly diagonal (off-diagonal above tolerance)
    mat4 = np.array([[1, 1e-3], [0, 2]])
    assert not is_diag(mat4)

    print("All tests passed.")

test_is_diag()


All tests passed.
